# Music in Python

## What is Sound?

## Making a Sound in Python

In convention, the $A$ above middle $C$ has a frequency of $440Hz$. Then we can derive any note using this formula:
$$f(n)=(\sqrt[12] 2)^{n-49}\times440\text{Hz}$$
where $n$ is the rank of the key ($A4$ is the $49^{th}$ key).

Below is the code to generate the frequency of 88 keys on the piano.

In [ ]:
import numpy as np

def get_piano_notes():
    # White keys are in Uppercase and black keys (sharps) are in lower case
    octave = ['C', 'c', 'D', 'd', 'E', 'F', 'f', 'G', 'g', 'A', 'a', 'B']
    base_freq = 440     # Frequency of Note A4
    keys = np.array([x + str(y) for y in range(0, 9) for x in octave])
    # Trim to standard 88 keys
    start = np.where(keys == 'A0')[0][0]
    end = np.where(keys == 'C8')[0][0]
    keys = keys[start : end + 1]

    note_freqs = dict(zip(keys, [2**((n+1-49)/12)*base_freq for n in range(len(keys))]))
    note_freqs[''] = 0.0    # stop
    return note_freqs

Now we want to make a wave of middle $C$:

In [ ]:
import numpy as np

def get_sine_wave(frequency, duration, sample_rate=44100, amplitude=4096):
    t = np.linspace(0, duration, int(sample_rate*duration)) # Time axis
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

Here `duration` is in number of seconds, `sample_rate` determines the quality of sound, and `amplitude` determines the volume.

Using the two functions above and `Scipy`'s `wavfile` module, we can create a `.wav` file of middle $C$.

In [ ]:
import numpy as np
from scipy.io import wavfile

# Get middle C frequency
note_freqs = get_piano_notes()
frequency_C4 = note_freqs['C4']

# Pure sine wave
sine_wave_C4 = get_sine_wave(frequency_C4, duration=2, amplitude=16384)
wavfile.write('pure_c.wav', rate=44100, data=sine_wave_C4.astype(np.int16))

Let's plot the wave of $C4$ and $D4$.

In [ ]:
import matplotlib.pyplot as plt

frequency_D4 = note_freqs['D4']
sine_wave_C4 = get_sine_wave(frequency_C4, duration=0.02, amplitude=16384)
sine_wave_D4 = get_sine_wave(frequency_D4, duration=0.02, amplitude=16384)

plt.plot(sine_wave_C4, label='C')
plt.plot(sine_wave_D4, label='D')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sine Wave of Middle C and D')
plt.show()

## Making Realistic Sound

Once again using Scipy, we will load the piano audio and plot the sound wave using Matplotlib.

In [ ]:
from scipy.io import wavfile
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')

# Load data from wav file
sample_rate, middle_c = wavfile.read('data/piano_c4.wav')

# Plot sound wave
plt.plot(middle_c[500:1500])
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Sound Wave of Middle C on Piano')
plt.grid()

Let's apply fast Fourier transform (FFT) on our piano audio and plot the spectrogram.

In [ ]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')

# Load data from wav file
sample_rate, middle_c = wavfile.read('data/piano_c4.wav')

#FFT
t = np.arange(middle_c.shape[0])
freq = np.fft.fftfreq(t.shape[-1])*sample_rate
sp0 = np.fft.fft(middle_c[:,0])
sp1 = np.fft.fft(middle_c[:,1])

# Plot spectrum
plt.plot(freq, abs(sp0.real))
plt.plot(freq, abs(sp1.real))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.title('Spectrum of Middle C Recording on Piano')
plt.xlim((0, 2000))
plt.grid()

The piece of code below calculates the ratio of the magnitude between each overtone and the fundamental from the piano sample so that we can apply that to our pure sine waves.

In [ ]:
import numpy as np

# Get positive frequency
idx = np.where(freq > 0)[0]
freq = freq[idx]
sp = sp1[idx]

# Get dominant frequencies
sort = np.argsort(-abs(sp.real))[:100]
dom_freq = freq[sort]

# Round and calculate amplitude ratio
freq_ratio = np.round(dom_freq/frequency_C4)
unique_freq_ratio = np.unique(freq_ratio)
amp_ratio = abs(sp.real[sort]/np.sum(sp.real[sort]))
factor = np.zeros((int(unique_freq_ratio[-1]), ))
for i in range(factor.shape[0]):
    idx = np.where(freq_ratio==i+1)[0]
    factor[i] = np.sum(amp_ratio[idx])
factor = factor/np.sum(factor)

Now we apply those ratios to our fundamental note and its overtones.

In [ ]:
def apply_overtones(frequency, duration, factor, sample_rate=44100, amplitude=4096):

    assert abs(1-sum(factor)) < 1e-8
    
    frequencies = np.minimum(np.array([frequency*(x+1) for x in range(len(factor))]), sample_rate//2)
    amplitudes = np.array([amplitude*x for x in factor])
    
    fundamental = get_sine_wave(frequencies[0], duration, sample_rate, amplitudes[0])
    for i in range(1, len(factor)):
        overtone = get_sine_wave(frequencies[i], duration, sample_rate, amplitudes[i])
        fundamental += overtone
    return fundamental

# Construct harmonic series
note = apply_overtones(frequency_C4, duration=2.5, factor=factor)
wavfile.write('overtone_c.wav', sample_rate, note.astype(np.int16))

When we press an actual piano key, it started out light before quickly getting louder, and the sound diminishes over time. One model to describe how the sound changes is ADSR (attack, decay, sustain and release). Essentially it describes sound as going through four stages: the initial stage of incline, then descending to a lower level, maintaining there for a little while before diminishing to zero. 

In [ ]:
import numpy as np

def get_adsr_weights(frequency, duration, length, decay, sustain_level, sample_rate=44100):

    assert abs(sum(length)-1) < 1e-8
    assert len(length) ==len(decay) == 4
    
    intervals = int(duration*frequency)
    len_A = np.maximum(int(intervals*length[0]),1)
    len_D = np.maximum(int(intervals*length[1]),1)
    len_S = np.maximum(int(intervals*length[2]),1)
    len_R = np.maximum(int(intervals*length[3]),1)
    
    decay_A = decay[0]
    decay_D = decay[1]
    decay_S = decay[2]
    decay_R = decay[3]
    
    A = 1/np.array([(1-decay_A)**n for n in range(len_A)])
    A = A/np.nanmax(A)
    D = np.array([(1-decay_D)**n for n in range(len_D)])
    D = D*(1-sustain_level)+sustain_level
    S = np.array([(1-decay_S)**n for n in range(len_S)])
    S = S*sustain_level
    R = np.array([(1-decay_R)**n for n in range(len_R)])
    R = R*S[-1]
    
    weights = np.concatenate((A,D,S,R))
    smoothing = np.array([0.1*(1-0.1)**n for n in range(5)])
    smoothing = smoothing/np.nansum(smoothing)
    weights = np.convolve(weights, smoothing, mode='same')
    
    weights = np.repeat(weights, int(sample_rate*duration/intervals))
    tail = int(sample_rate*duration-weights.shape[0])
    if tail > 0:
        weights = np.concatenate((weights, weights[-1]-weights[-1]/tail*np.arange(tail)))
    return weights

## Putting It Together

In [ ]:
import numpy as np
from scipy.io import wavfile

# Get sound wave
note = apply_overtones(frequency_C4, duration=2.5, factor=factor)

# Apply smooth ADSR weights
weights = get_adsr_weights(frequency_C4, duration=2.5, length=[0.05, 0.25, 0.55, 0.15], decay=[0.075,0.02,0.005,0.1], sustain_level=0.1)

# Write to file
data = note*weights
data = data*(16384/np.max(data)) # Adjusting the Amplitude 
wavfile.write('synthetic_c.wav', sample_rate, data.astype(np.int16))

## A version to access the note with different duration

In [ ]:
def n(note, duration=1.0, amplitude=16384):
    freq = note_freqs[note];
    note = apply_overtones(freq, duration=duration, factor=factor)
    weights = get_adsr_weights(freq, duration=duration, length=[0.05,0.25,0.55,0.15], decay=[0.075,0.02,0.005,0.1], sustain_level=0.1)
    data = note*weights
    return data

In [ ]:
score = np.concatenate([n('C4',.5)+n('C3',.5),n('C4',.5)+n('C3',.5),n('G4',.5)+n('G3',.5),n('G4',.5)+n('G3',.5),
        n('A4',.5)+n('A3',.5),n('A4',.5)+n('A3',.5),n('G4',1)+n('G3',1),
        n('F4',.5)+n('F3',.5),n('F4',.5)+n('F3',.5),n('E4',.5)+n('E3',.5),n('E4',.25)+n('E3',.25),n('E4',.25)+n('E3',.25),
        n('D4',.5)+n('D3',.5),n('D4',.5)+n('D3',.5),n('C4',1)+n('C3',1)])
wavfile.write('twinkle_twinkle_little_star.wav', sample_rate, score.astype(np.int16))